Przykład Retreival-Augumented Generation (RAG) z użyciem tekstu ze stron internetowych  
Repozytorium z modelami otwartoźródłowymi: https://huggingface.co/models

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import requests

def save_html_from_dict(urls_and_filenames):
    retrieved_files = []
    for pair in urls_and_filenames:
        url = pair['url']
        file_name = pair['file_name']

        response = requests.get(url)

        if response.status_code == 200:
            html_content = response.text

            with open(file_name, "w", encoding="utf-8") as file:
                file.write(html_content)

            retrieved_files.append(file_name)
            print("HTML content from", url, "saved to", file_name)
        else:
            print("Failed to fetch content from", url)

    return retrieved_files

# URLs and corresponding file names in a list of dictionaries
urls_and_filenames = [
    {'url': 'YOUR_URL_HERE', 'file_name': 'output1.html'},
    {'url': 'YOUR_URL_HERE', 'file_name': 'output2.html'},
    {'url': 'YOUR_URL_HERE', 'file_name': 'output3.html'},
]

# Save HTML content from multiple URLs using the dictionary list
retrieved_files = save_html_from_dict(urls_and_filenames)

print("Retrieved files:", retrieved_files)


In [ ]:

from langchain_community.document_loaders import UnstructuredHTMLLoader

loaded_data = []

for path in retrieved_files:
    loader = UnstructuredHTMLLoader(file_path=path)
    data = loader.load()
    loaded_data.extend(data)
loaded_data


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200)

docs = text_splitter.split_documents(loaded_data)
docs

In [ ]:
import lancedb
from langchain.embeddings.openai import OpenAIEmbeddings

db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "my_table",
    data=[
        {
            "vector": OpenAIEmbeddings().embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

Embeddings: https://platform.openai.com/docs/guides/embeddings/what-are-embeddings  
Wizualizacja embeddingów dla danych z Twittera https://github.com/nomic-ai/nomic

In [ ]:
from langchain.vectorstores.lancedb import LanceDB
docsearch = LanceDB.from_documents(docs, OpenAIEmbeddings(), connection=table)


docs_similarity = docsearch.similarity_search(query='Jakie są wymagania dla PHP web developera', k=3)
docs_similarity

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

prompt = PromptTemplate(
    input_variables=["question", "docs"],
    template="""
    Jesteś pomocnym asystentem, który potrafi odpowiadać na pytania dotyczące zadań, wymagań i mile widzianych umiejętności w ofertach pracy na stronach internetowych.

    Odpowiedz na następujące pytanie: {question}
    Szukając następującej strony internetowej: {docs}

    Wykorzystaj tylko faktyczne informacje z transkryptu, aby odpowiedzieć na pytanie.

    Jeśli uważasz, że nie masz wystarczających informacji, aby odpowiedzieć na pytanie, powiedz "Nie wiem".

    Twoje odpowiedzi powinny być obszerne i szczegółowe.
    """,
)

chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

In [ ]:
question="Czy PHP/JS developer musi lubić ogórki kiszone"

docs_similarity = docsearch.similarity_search(query=question, k=3)
docs_similarity

In [ ]:

docs_page_content = " " .join([doc.page_content for doc in docs_similarity])
docs_page_content

In [ ]:

response = chain.run(question=question, docs=docs_page_content)
print(response)